In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Introduction
- We use three Keras functionalities (inherits from keras model) to train, evaluate and pedict from the model that developed
    - Model.fit() - for training
    - Model.evaluate() - to evaluate the model
    - Model.predict() - to infere the model on test example or test dataset
- When we use built in loops for training and evaluation, process will be same for both Seqential and Functional API models

## A First end -to-end Example
- Data can be fed to training loops either using
    - Numpy Arrays (When the data is small and can be fit into memory)
    - tf.data Dataset objects

Lets consider the following model for MNIST classification:

A Typical end-to-end workflow looks like consists of:
- Training
- Validation on Hold out Data (generated from original training data)
- Evaluation on Test Data


In [2]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense1")(inputs)
x = layers.Dense(64, activation="relu", name="dense2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictio")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1,784).astype("float32")/255
x_test = x_test.reshape(-1,784).astype("float32")/255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Specify the training configuration (optimizser, loss, metrics)
model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = [keras.metrics.SparseCategoricalAccuracy()],)

# fit() is for training the model with several parameters
print("Training The Model...")
history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_data=(x_val, y_val))

Training The Model...
Epoch 1/2
782/782 [==============================] - 6s 6ms/step - loss: 0.3463 - sparse_categorical_accuracy: 0.9022 - val_loss: 0.1880 - val_sparse_categorical_accuracy: 0.9441
Epoch 2/2
782/782 [==============================] - 5s 7ms/step - loss: 0.1593 - sparse_categorical_accuracy: 0.9526 - val_loss: 0.1666 - val_sparse_categorical_accuracy: 0.9514


In [4]:
# history object holds the metrics and losses for each epoch of both training and validation data
history.history

{'loss': [0.34626641869544983, 0.1593426764011383],
 'sparse_categorical_accuracy': [0.902239978313446, 0.9526000022888184],
 'val_loss': [0.18800930678844452, 0.16658279299736023],
 'val_sparse_categorical_accuracy': [0.944100022315979, 0.9513999819755554]}

In [5]:
# Evaluate the model on the test data using "evaluate" method
print("Evauae on Test Data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)
# Get predictions on individual images or batch of images using predict method
print("Generate Predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape", predictions.shape)

Evauae on Test Data
79/79 [==============================] - 1s 8ms/step - loss: 0.1677 - sparse_categorical_accuracy: 0.9495
test loss, test acc: [0.16769881546497345, 0.9495000243186951]
Generate Predictions for 3 samples
1/1 [==============================] - 0s 249ms/step
predictions shape (3, 10)


## Compiling a Model: Loss, Metrics, Optimizer
To train a model before going to fit() we need to compile the model with following fields
- optimizer - Algorithm for Backpropogation (example: Adam, RMSProp, Adagrad,...etc)
- loss - If the model have multiple outputs then we can specify different loss functions for each output
- metrics - its list of where we can specify any number of metrics. and also for multi output model we can specify multiple types of metrics

If we want to go with default values for (optimizer, loss and metrics) we can specify them in strings. if we want to customize them we need to call respective functions from keras


In [6]:
# Model compilation with default fileds
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [7]:
# Model compilation with custom functions
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [8]:
# For later reuse, let's put our model definition and compile step in functions; we will call them several times across different examples in this guide.
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

### Builtin Optimizers, Losses and Metircs
Optimizers: tf.keras.optimizers.
- SGD() (with or without momentum)
- RMSProp()
- Adam()
- Adagard()
- Adadelta()
- Adamax()

Losses: tf.keras.losses.
- BinaryCrossentropy()
- CategoricalCrossentropy()
- CategoricalHinge()
- CosineSimilairty()
- Hinge()
- KLDivergence()
- MeanAbsoluteError()
- MeanAbsolutePercentageError()
- MeanSquaredError()
- MeanSquaredLogarithmicError()
- SparseCategoricalCrossentropy()

Metrics: tf.keras.metrics.
- AUC()
- Precision()
- Recall()
- Accuracy()

Apart from there if we want to create a custom functions Keras has the feasability to create

### Custom Loss:
To create a custom loss function we can do it in two ways
- create a function which takes y_true and y_pred as inputs (it wont accept other inputs)
- if we want to have other paramters along with y_true and y_pred we need to create a custom loss class inherited from tf.keras.losses.Loss



In [9]:
# The following example shows a loss function that computes the mean squared error between the real data and the predictions:
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred))


model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

# We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 6s 6ms/step - loss: 0.0157


If you need a loss function that takes in parameters beside y_true and y_pred, you can subclass the tf.keras.losses.Loss class and implement the following two methods:
- __init__(self): accept parameters to pass during the call of your loss function
- call(self, y_true, y_pred): use the targets (y_true) and the model predictions (y_pred) to compute the model's loss

Let's say you want to use mean squared error, but with an added term that will de-incentivize prediction values far from 0.5 (we assume that the categorical targets are one-hot encoded and take values between 0 and 1). This creates an incentive for the model not to be too confident, which may help reduce overfitting (we won't know if it works until we try!).

Here's how you would do it:

In [10]:
class CustomMSE(tf.keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super(CustomMSE, self).__init__(name=name)
        self.regularization_factor = regularization_factor
    def call(self, y_true, y_pred):
        mse = tf.reduce_mean(tf.square(y_true-y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse+reg*self.regularization_factor

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)


782/782 [==============================] - 8s 7ms/step - loss: 0.0385


### Custom Metrics:
